# Aggregate Summary of Statistics and Make a Baseline Table

This notebook aggregates summary of statistics of all infection-related hospital admission data and processes them to make an aggregated baseline table.

In [1]:
import pandas as pd
import numpy as np

## 0- functions and read data

In [8]:
summary_table_lrti_no_abs_incdt = pd.read_csv(f'../output/hospitalisation_prediction_lrti/summary_table_lrti_no_abs_incdt.csv')
summary_table_lrti_abs_incdt = pd.read_csv(f'../output/hospitalisation_prediction_lrti/summary_table_lrti_abs_incdt.csv')
summary_table_lrti_no_abs_prevl = pd.read_csv(f'../output/hospitalisation_prediction_lrti/summary_table_lrti_no_abs_prevl.csv')
summary_table_lrti_abs_prevl = pd.read_csv(f'../output/hospitalisation_prediction_lrti/summary_table_lrti_abs_prevl.csv')
summary_table_urti_combined_no_abs_incdt = pd.read_csv(f'../output/hospitalisation_prediction_urti_combined/summary_table_urti_combined_no_abs_incdt.csv')
summary_table_urti_combined_abs_incdt = pd.read_csv(f'../output/hospitalisation_prediction_urti_combined/summary_table_urti_combined_abs_incdt.csv')
summary_table_urti_combined_no_abs_prevl = pd.read_csv(f'../output/hospitalisation_prediction_urti_combined/summary_table_urti_combined_no_abs_prevl.csv')
summary_table_urti_combined_abs_prevl = pd.read_csv(f'../output/hospitalisation_prediction_urti_combined/summary_table_urti_combined_abs_prevl.csv')
summary_table_uti_no_abs_incdt = pd.read_csv(f'../output/hospitalisation_prediction_uti/summary_table_uti_no_abs_incdt.csv')
summary_table_uti_abs_incdt = pd.read_csv(f'../output/hospitalisation_prediction_uti/summary_table_uti_abs_incdt.csv')
summary_table_uti_no_abs_prevl = pd.read_csv(f'../output/hospitalisation_prediction_uti/summary_table_uti_no_abs_prevl.csv')
summary_table_uti_abs_prevl = pd.read_csv(f'../output/hospitalisation_prediction_uti/summary_table_uti_abs_prevl.csv')
summary_table_sinusitis_no_abs_incdt = pd.read_csv(f'../output/hospitalisation_prediction_sinusitis/summary_table_sinusitis_no_abs_incdt.csv')
summary_table_sinusitis_abs_incdt = pd.read_csv(f'../output/hospitalisation_prediction_sinusitis/summary_table_sinusitis_abs_incdt.csv')
summary_table_sinusitis_no_abs_prevl = pd.read_csv(f'../output/hospitalisation_prediction_sinusitis/summary_table_sinusitis_no_abs_prevl.csv')
summary_table_sinusitis_abs_prevl = pd.read_csv(f'../output/hospitalisation_prediction_sinusitis/summary_table_sinusitis_abs_prevl.csv')
summary_table_ot_externa_no_abs_incdt = pd.read_csv(f'../output/hospitalisation_prediction_ot_externa/summary_table_ot_externa_no_abs_incdt.csv')
summary_table_ot_externa_abs_incdt = pd.read_csv(f'../output/hospitalisation_prediction_ot_externa/summary_table_ot_externa_abs_incdt.csv')
summary_table_ot_externa_no_abs_prevl = pd.read_csv(f'../output/hospitalisation_prediction_ot_externa/summary_table_ot_externa_no_abs_prevl.csv')
summary_table_ot_externa_abs_prevl = pd.read_csv(f'../output/hospitalisation_prediction_ot_externa/summary_table_ot_externa_abs_prevl.csv')
summary_table_otmedia_no_abs_incdt = pd.read_csv(f'../output/hospitalisation_prediction_otmedia/summary_table_otmedia_no_abs_incdt.csv')
summary_table_otmedia_abs_incdt = pd.read_csv(f'../output/hospitalisation_prediction_otmedia/summary_table_otmedia_abs_incdt.csv')
summary_table_otmedia_no_abs_prevl = pd.read_csv(f'../output/hospitalisation_prediction_otmedia/summary_table_otmedia_no_abs_prevl.csv')
summary_table_otmedia_abs_prevl = pd.read_csv(f'../output/hospitalisation_prediction_otmedia/summary_table_otmedia_abs_prevl.csv')
summary_table_urti_no_abs_incdt = pd.read_csv(f'../output/hospitalisation_prediction_urti/summary_table_urti_no_abs_incdt.csv')
summary_table_urti_abs_incdt = pd.read_csv(f'../output/hospitalisation_prediction_urti/summary_table_urti_abs_incdt.csv')
summary_table_urti_no_abs_prevl = pd.read_csv(f'../output/hospitalisation_prediction_urti/summary_table_urti_no_abs_prevl.csv')
summary_table_urti_abs_prevl = pd.read_csv(f'../output/hospitalisation_prediction_urti/summary_table_urti_abs_prevl.csv')
summary_table_cough_no_abs_incdt = pd.read_csv(f'../output/hospitalisation_prediction_cough/summary_table_cough_no_abs_incdt.csv')
summary_table_cough_abs_incdt = pd.read_csv(f'../output/hospitalisation_prediction_cough/summary_table_cough_abs_incdt.csv')
summary_table_cough_no_abs_prevl = pd.read_csv(f'../output/hospitalisation_prediction_cough/summary_table_cough_no_abs_prevl.csv')
summary_table_cough_abs_prevl = pd.read_csv(f'../output/hospitalisation_prediction_cough/summary_table_cough_abs_prevl.csv')
summary_table_cough_cold_no_abs_incdt = pd.read_csv(f'../output/hospitalisation_prediction_cough_cold/summary_table_cough_cold_no_abs_incdt.csv')
summary_table_cough_cold_abs_incdt = pd.read_csv(f'../output/hospitalisation_prediction_cough_cold/summary_table_cough_cold_abs_incdt.csv')
summary_table_cough_cold_no_abs_prevl = pd.read_csv(f'../output/hospitalisation_prediction_cough_cold/summary_table_cough_cold_no_abs_prevl.csv')
summary_table_cough_cold_abs_prevl = pd.read_csv(f'../output/hospitalisation_prediction_cough_cold/summary_table_cough_cold_abs_prevl.csv')
summary_table_throat_no_abs_incdt = pd.read_csv(f'../output/hospitalisation_prediction_throat/summary_table_throat_no_abs_incdt.csv')
summary_table_throat_abs_incdt = pd.read_csv(f'../output/hospitalisation_prediction_throat/summary_table_throat_abs_incdt.csv')
summary_table_throat_no_abs_prevl = pd.read_csv(f'../output/hospitalisation_prediction_throat/summary_table_throat_no_abs_prevl.csv')
summary_table_throat_abs_prevl = pd.read_csv(f'../output/hospitalisation_prediction_throat/summary_table_throat_abs_prevl.csv')

In [9]:
summary_table_lrti_abs_incdt

,variable,count,mean,std
0,age_cat_15_24,2370.0,0.148612,0.355717
1,age_cat_25_34,2170.0,0.135894,0.342686
2,age_cat_35_44,2345.0,0.146858,0.353975
3,age_cat_45_54,2290.0,0.143349,0.350440
4,age_cat_55_64,2270.0,0.142096,0.349160
5,age_cat_65_74,2330.0,0.145856,0.352972
6,age_cat_75_more,2190.0,0.137335,0.344211
7,sex_female,8055.0,0.504542,0.499995
8,sex_male,7910.0,0.495458,0.499995
9,CCI_cat_high,1635.0,0.102500,0.303314


In [10]:
print(summary_table_lrti_abs_incdt['variable'].tolist())

['age_cat_15_24', 'age_cat_25_34', 'age_cat_35_44', 'age_cat_45_54', 'age_cat_55_64', 'age_cat_65_74', 'age_cat_75_more', 'sex_female', 'sex_male', 'CCI_cat_high', 'CCI_cat_low', 'CCI_cat_medium', 'CCI_cat_very_high', 'CCI_cat_very_low', 'flu_vaccine_no', 'flu_vaccine_yes', 'bmi_cat_healthy_weight', 'bmi_cat_obese', 'bmi_cat_overweight', 'bmi_cat_underweight', 'bmi_cat_unknown', 'region_east', 'region_east_midlands', 'region_london', 'region_north_east', 'region_north_west', 'region_south_east', 'region_south_west', 'region_west_midlands', 'region_yorkshire', 'imd_affluent', 'imd_medium', 'imd_unaffluent', 'imd_unknown', 'imd_very_affluent', 'imd_very_unaffluent', 'ethnicity_asian', 'ethnicity_other', 'ethnicity_unknown', 'ethnicity_white', 'smoking_status_ex_smoker', 'smoking_status_never_smoked', 'smoking_status_smoker', 'smoking_status_unknown', 'season_autumn', 'season_spring', 'season_summer', 'season_winter', 'period_during_pandemic', 'period_post_2nd_lockdown', 'period_prepandem

In [4]:
# summary_table_throat_prevl

In [ ]:
summary_table_lrti_abs_incdt

In [11]:
def data_process(data):
    # #replace ab_date with ab_binary in variable column
    # data.loc[data['variable'].str.contains('_ab_date'), 'variable'] = 'ab_binary'
    #replace SMs with 5 in data
    for row in range(0, len(data)):
        if data['count'][row] == 'SM':
            data['count'][row] = 5
    #conver counts into int
    data['count_int'] = data['count'].astype(int)#.astype(str).replace('\.0', '', regex=True).astype(int)
    #convert means into percentages with two decimals
    data['mean_perc'] = data['mean']*100
    data['mean_perc'] = np.round(data['mean_perc'], decimals=2)
    #custom order of variables to fit tables of the manuscript
    data['variable'] = pd.Categorical(data['variable'], ['age_cat_15_24', 'age_cat_25_34', 'age_cat_35_44', 'age_cat_45_54', 'age_cat_55_64', 'age_cat_65_74', 'age_cat_75_more', 'sex_female', 'sex_male', 'CCI_cat_high', 'CCI_cat_low', 'CCI_cat_medium', 'CCI_cat_very_high', 'CCI_cat_very_low', 'flu_vaccine_no', 'flu_vaccine_yes', 'bmi_cat_healthy_weight', 'bmi_cat_obese', 'bmi_cat_overweight', 'bmi_cat_underweight', 'bmi_cat_unknown', 'region_east', 'region_east_midlands', 'region_london', 'region_north_east', 'region_north_west', 'region_south_east', 'region_south_west', 'region_west_midlands', 'region_yorkshire', 'imd_affluent', 'imd_medium', 'imd_unaffluent', 'imd_unknown', 'imd_very_affluent', 'imd_very_unaffluent', 'ethnicity_asian', 'ethnicity_other', 'ethnicity_unknown', 'ethnicity_white', 'smoking_status_ex_smoker', 'smoking_status_never_smoked', 'smoking_status_smoker', 'smoking_status_unknown', 'season_autumn', 'season_spring', 'season_summer', 'season_winter', 'period_during_pandemic', 'period_post_2nd_lockdown', 'period_prepandemic', 'antibacterial_brit'])
    data = data.sort_values('variable')
    #keep mean values of gp_count and antibacterial_brit
    data['mean'] = np.round(data['mean'], decimals=2)
    #reset index to prevent issue in calculating total value
    data = data.reset_index(drop=True)
    #calculate total from sex_male and sex_female
    total = data.iloc[data[data['variable']=='sex_female'].index[0],data.columns.get_loc('count')] + data.iloc[data[data['variable']=='sex_male'].index[0],data.columns.get_loc('count')]
    #create a new dataframe for the new row and add comma to the total's thousands
    new_row = pd.DataFrame({'variable':'Total', 'count':'-', 'mean':'-', 'std':'-', 'count_int':'-', 'mean_perc':'-', 'count_mean':total}, index=[0])
    new_row['count_mean'] = new_row['count_mean'].astype(int)
    new_row['count_mean'] = new_row['count_mean'].apply(lambda x : "{:,}".format(x))
    #keep 2 digits of mean_perc
    data['mean_perc'] = data['mean_perc'].apply(lambda x: "{:.2f}".format(x))
    #add comma after 3 digits
    data['count_int'] = data['count_int'].astype(int)
    data['count_int'] = data['count_int'].apply(lambda x : "{:,}".format(x))
    #replace SN
    for row in range(0, len(data)):
        if data['count'][row] == 5:
            data['count_int'][row] = 'SN'
            data['mean_perc'][row] = 'SN'
    #combine counts and means
    data['count_mean'] = data['count_int'].astype(str) + ' (' + data['mean_perc'].astype(str) + ')'
    #append row to the dataframe
    data = pd.concat([data, new_row])
    #replace
    data['std_round'] = pd.to_numeric(data['std'],errors='coerce')
    data['std_round'] = np.round(data['std_round'], decimals=2)
    data.iloc[int(data[data['variable']=='antibacterial_brit'].index[0]),int(data.columns.get_loc('count_mean'))] = str(data.iloc[int(data[data['variable']=='antibacterial_brit'].index[0]),int(data.columns.get_loc('mean'))]) + ' (' + str(data.iloc[int(data[data['variable']=='antibacterial_brit'].index[0]),int(data.columns.get_loc('std_round'))]) + ')'

    return data

## 1- process data to make basline table, and save it

In [12]:
summary_table_lrti_no_abs_incdt = data_process(summary_table_lrti_no_abs_incdt)
summary_table_lrti_abs_incdt = data_process(summary_table_lrti_abs_incdt)
summary_table_lrti_no_abs_prevl = data_process(summary_table_lrti_no_abs_prevl)
summary_table_lrti_abs_prevl = data_process(summary_table_lrti_abs_prevl)
# summary_table_urti_combined_no_abs_incdt = data_process(summary_table_urti_combined_no_abs_incdt)
# summary_table_urti_combined_abs_incdt = data_process(summary_table_urti_combined_abs_incdt)
# summary_table_urti_combined_no_abs_prevl = data_process(summary_table_urti_combined_no_abs_prevl)
# summary_table_urti_combined_abs_prevl = data_process(summary_table_urti_combined_abs_prevl)
# summary_table_uti_no_abs_incdt = data_process(summary_table_uti_no_abs_incdt)
# summary_table_uti_abs_incdt = data_process(summary_table_uti_abs_incdt)
# summary_table_uti_no_abs_prevl = data_process(summary_table_uti_no_abs_prevl)
# summary_table_uti_abs_prevl = data_process(summary_table_uti_abs_prevl)
# summary_table_sinusitis_no_abs_incdt = data_process(summary_table_sinusitis_no_abs_incdt)
# summary_table_sinusitis_abs_incdt = data_process(summary_table_sinusitis_abs_incdt)
# summary_table_sinusitis_no_abs_prevl = data_process(summary_table_sinusitis_no_abs_prevl)
# summary_table_sinusitis_abs_prevl = data_process(summary_table_sinusitis_abs_prevl)
# summary_table_ot_externa_no_abs_incdt = data_process(summary_table_ot_externa_no_abs_incdt)
# summary_table_ot_externa_abs_incdt = data_process(summary_table_ot_externa_abs_incdt)
# summary_table_ot_externa_no_abs_prevl = data_process(summary_table_ot_externa_no_abs_prevl)
# summary_table_ot_externa_abs_prevl = data_process(summary_table_ot_externa_abs_prevl)
# summary_table_otmedia_no_abs_incdt = data_process(summary_table_otmedia_no_abs_incdt)
# summary_table_otmedia_abs_incdt = data_process(summary_table_otmedia_abs_incdt)
# summary_table_otmedia_no_abs_prevl = data_process(summary_table_otmedia_no_abs_prevl)
# summary_table_otmedia_abs_prevl = data_process(summary_table_otmedia_abs_prevl)
# summary_table_urti_no_abs_incdt = data_process(summary_table_urti_no_abs_incdt)
# summary_table_urti_abs_incdt = data_process(summary_table_urti_abs_incdt)
# summary_table_urti_no_abs_prevl = data_process(summary_table_urti_no_abs_prevl)
# summary_table_urti_abs_prevl = data_process(summary_table_urti_abs_prevl)
# summary_table_cough_no_abs_incdt = data_process(summary_table_cough_no_abs_incdt)
# summary_table_cough_abs_incdt = data_process(summary_table_cough_abs_incdt)
# summary_table_cough_no_abs_prevl = data_process(summary_table_cough_no_abs_prevl)
# summary_table_cough_abs_prevl = data_process(summary_table_cough_abs_prevl)
# summary_table_cough_cold_no_abs_incdt = data_process(summary_table_cough_cold_no_abs_incdt)
# summary_table_cough_cold_abs_incdt = data_process(summary_table_cough_cold_abs_incdt)
# summary_table_cough_cold_no_abs_prevl = data_process(summary_table_cough_cold_no_abs_prevl)
# summary_table_cough_cold_abs_prevl = data_process(summary_table_cough_cold_abs_prevl)
# summary_table_throat_no_abs_incdt = data_process(summary_table_throat_no_abs_incdt)
# summary_table_throat_abs_incdt = data_process(summary_table_throat_abs_incdt)
# summary_table_throat_no_abs_prevl = data_process(summary_table_throat_no_abs_prevl)
# summary_table_throat_abs_prevl = data_process(summary_table_throat_abs_prevl)

In [13]:
d = {'variable': summary_table_lrti_no_abs_incdt['variable'],
     
     'lrti_no_abs_incdt': summary_table_lrti_no_abs_incdt['count_mean'],
     'lrti_abs_incdt': summary_table_lrti_abs_incdt['count_mean'],
     'lrti_no_abs_prevl': summary_table_lrti_no_abs_prevl['count_mean'],
     'lrti_abs_prevl': summary_table_lrti_abs_prevl['count_mean'],

     'urti_combined_no_abs_incdt': summary_table_urti_combined_no_abs_incdt['count_mean'],
     'urti_combined_abs_incdt': summary_table_urti_combined_abs_incdt['count_mean'],
     'urti_combined_no_abs_prevl': summary_table_urti_combined_no_abs_prevl['count_mean'],
     'urti_combined_abs_prevl': summary_table_urti_combined_abs_prevl['count_mean'],

    'uti_no_abs_incdt': summary_table_uti_no_abs_incdt['count_mean'],
    'uti_abs_incdt': summary_table_uti_abs_incdt['count_mean'],
    'uti_no_abs_prevl': summary_table_uti_no_abs_prevl['count_mean'],
    'uti_abs_prevl': summary_table_uti_abs_prevl['count_mean'],

    'sinusitis_no_abs_incdt': summary_table_sinusitis_no_abs_incdt['count_mean'],
    'sinusitis_abs_incdt': summary_table_sinusitis_abs_incdt['count_mean'],
    'sinusitis_no_abs_prevl': summary_table_sinusitis_no_abs_prevl['count_mean'],
    'sinusitis_abs_prevl': summary_table_sinusitis_abs_prevl['count_mean'],

    'ot_externa_no_abs_incdt': summary_table_ot_externa_no_abs_incdt['count_mean'],
    'ot_externa_abs_incdt': summary_table_ot_externa_abs_incdt['count_mean'],
    'ot_externa_no_abs_prevl': summary_table_ot_externa_no_abs_prevl['count_mean'],
    'ot_externa_abs_prevl': summary_table_ot_externa_abs_prevl['count_mean'],

    'otmedia_no_abs_incdt': summary_table_otmedia_no_abs_incdt['count_mean'],
    'otmedia_abs_incdt': summary_table_otmedia_abs_incdt['count_mean'],
    'otmedia_no_abs_prevl': summary_table_otmedia_no_abs_prevl['count_mean'],
    'otmedia_abs_prevl': summary_table_otmedia_abs_prevl['count_mean'],

    'urti_no_abs_incdt': summary_table_urti_no_abs_incdt['count_mean'],
    'urti_abs_incdt': summary_table_urti_abs_incdt['count_mean'],
    'urti_no_abs_prevl': summary_table_urti_no_abs_prevl['count_mean'],
    'urti_abs_prevl': summary_table_urti_abs_prevl['count_mean'],

    'cough_no_abs_incdt': summary_table_cough_no_abs_incdt['count_mean'],
    'cough_abs_incdt': summary_table_cough_abs_incdt['count_mean'],
    'cough_no_abs_prevl': summary_table_cough_no_abs_prevl['count_mean'],
    'cough_abs_prevl': summary_table_cough_abs_prevl['count_mean'],

    'cough_cold_no_abs_incdt': summary_table_cough_cold_no_abs_incdt['count_mean'],
    'cough_cold_abs_incdt': summary_table_cough_cold_abs_incdt['count_mean'],
    'cough_cold_no_abs_prevl': summary_table_cough_cold_no_abs_prevl['count_mean'],
    'cough_cold_abs_prevl': summary_table_cough_cold_abs_prevl['count_mean'],

    'throat_no_abs_incdt': summary_table_throat_no_abs_incdt['count_mean'],
    'throat_abs_incdt': summary_table_throat_abs_incdt['count_mean'],
    'throat_no_abs_prevl': summary_table_throat_no_abs_prevl['count_mean'],
    'throat_abs_prevl': summary_table_throat_abs_prevl['count_mean'],
     
    }
data = pd.DataFrame(data=d)

data.to_csv(f'../output/aggregate_tables/baseline_table_all_infections.csv', index=False)
data

,variable,lrti_no_abs_incdt,lrti_abs_incdt,lrti_no_abs_prevl,lrti_abs_prevl
0,age_cat_15_24,"20,495 (14.20)","2,370 (14.86)","2,325 (14.50)",250 (13.80)
1,age_cat_25_34,"20,440 (14.17)","2,170 (13.59)","2,285 (14.23)",250 (13.96)
2,age_cat_35_44,"20,495 (14.20)","2,345 (14.69)","2,300 (14.33)",260 (14.40)
3,age_cat_45_54,"20,500 (14.21)","2,290 (14.33)","2,260 (14.08)",250 (13.85)
4,age_cat_55_64,"20,745 (14.38)","2,270 (14.21)","2,300 (14.34)",250 (13.74)
5,age_cat_65_74,"20,940 (14.51)","2,330 (14.59)","2,365 (14.73)",285 (15.73)
6,age_cat_75_more,"20,690 (14.34)","2,190 (13.73)","2,215 (13.79)",260 (14.52)
7,sex_female,"73,450 (50.90)","8,055 (50.45)","8,090 (50.41)",935 (51.91)
8,sex_male,"70,850 (49.10)","7,910 (49.55)","7,960 (49.59)",870 (48.09)
9,CCI_cat_high,"14,880 (10.31)","1,635 (10.25)","1,710 (10.65)",185 (10.19)
